In [1]:
pip install scx>=1.0.7

Note: you may need to restart the kernel to use updated packages.


<h1>Blinky P4</h1>
<p>A continuation of P3</p>
<p>You are considering adding a distribution center (DC) to your network. The transportation cost from the Assembly plants to this new DC is (USD)0.04 (inbound) and from DC to regions is (USD)0.08 (outbound). Note that the DC cannot keep any inventory. <b></b></p>
<p><b>Table 3: Distance in Miles - Assembly plants to DC</b></p>
<table width="570" height="137" style="width: 26.3852%; height: 68px;">
<tbody>
<tr style="height: 39px;">
<td width="20%" style="text-align: center; border: 1px solid black; width: 66.4133%; height: 39px;">Miles</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 33.0794%; height: 39px;">DC</td>
</tr>
<tr style="height: 39px;">
<td width="15%" style="text-align: center; border: 1px solid black; width: 66.4133%; height: 39px;">Assembly Plant 1</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 33.0794%; height: 39px;">190</td>
</tr>
<tr style="height: 39px;">
<td width="15%" style="text-align: center; border: 1px solid black; width: 66.4133%; height: 39px;">Assembly Plant 2</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 33.0794%; height: 39px;">150</td>
</tr>
</tbody>
</table>
<p><b>Table 4: Distance in Miles - DC to Regions</b></p>
<table width="1283" height="79" style="width: 36.2587%;">
<tbody>
<tr>
<td width="20%" style="text-align: center; border: 1px solid black; width: 40.1816%;">Miles</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.7069%;">Region 1</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.7069%;">Region 2</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.7069%;">Region 3</td>
</tr>
<tr>
<td width="20%" style="text-align: center; border: 1px solid black; width: 40.1816%;">DC</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.7069%;">15</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.7069%;">59</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.7069%;">79</td>
</tr>
</tbody>
</table>
<p><strong></strong></p>
<p><strong>Adjust your model considering the newly available information. Your goal is to minimize the total transportation cost.</strong></p>
<a href="https://youtu.be/3Ihka255Kus">Blinky P4 Video Walkthrough Link</a><br/>
<iframe width="560" height="315"
    src="//www.youtube.com/embed/3Ihka255Kus" frameborder="0" allowfullscreen>
</iframe>

In [2]:
from scx.optimize import Model

In [3]:
transport = [
    {
        'origin_name':'A1',
        'destination_name':'DC1',
        'distance': 190,
        'cost_per_mile':0.04,
    },
    {
        'origin_name':'A2',
        'destination_name':'DC1',
        'distance': 150,
        'cost_per_mile':0.04,
    },
    {
        'origin_name':'DC1',
        'destination_name':'R1',
        'distance': 15,
        'cost_per_mile':0.08,
    },
    {
        'origin_name':'DC1',
        'destination_name':'R2',
        'distance': 59,
        'cost_per_mile':0.08,
    },
    {
        'origin_name':'DC1',
        'destination_name':'R3',
        'distance': 79,
        'cost_per_mile':0.08,
    },
]

demand = [
    {
        'name':'R1', 
        'demand':2500
    },
    {
        'name':'R2', 
        'demand':4350
    },
    {
        'name':'R3', 
        'demand':3296
    },
]

assembly = [
    {
        'name':'A1',
        'limit':13000,
    },
    {
        'name':'A2',
        'limit':3500,
    },
]

distribution_center = [
    {
        'name': 'DC1'
    }
]

In [4]:
for t in transport:
    # Create decision variables for each item in transport
    t['amt']=Model.variable(name=f"{t['origin_name']}__{t['destination_name']}__amt", lowBound=0)
    # Calculate the variable cost of shipping for each item in tranport
    t['cost']=t['distance']*t['cost_per_mile']

In [5]:
# Initialize the my_model
my_model = Model(name="Blinky22", sense='minimize')


# Add the Objective Fn
my_model.add_objective(
    fn=Model.sum([t['amt']*t['cost'] for t in transport])
)

# Add Constraints
## Demand Constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}__demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination_name']==d['name']]) >= d['demand']
    )

## Supply Constraint
for a in assembly:
    my_model.add_constraint(
        name=f"{a['name']}__assembly_supply",
        fn=Model.sum([t['amt'] for t in transport if t['origin_name']==a['name']]) <= a['limit']
    )

## Conservation of Flow Constraint
for dc in distribution_center:
    my_model.add_constraint(
        name=f"{dc['name']}__conservation_of_flow",
        # Set inbound flows for the DC equal to outbound flows
        fn=Model.sum(
            [t['amt'] for t in transport if t['destination_name']==dc['name']]
        ) == Model.sum(
            [t['amt'] for t in transport if t['origin_name']==dc['name']]
        )
    )

# Solve the my_model
my_model.solve()

In [6]:
my_model.show_formulation()

Blinky22:
MINIMIZE
7.6000000000000005*A1__DC1__amt + 6.0*A2__DC1__amt + 1.2*DC1__R1__amt + 4.72*DC1__R2__amt + 6.32*DC1__R3__amt + 0.0
SUBJECT TO
R1__demand: DC1__R1__amt >= 2500

R2__demand: DC1__R2__amt >= 4350

R3__demand: DC1__R3__amt >= 3296

A1__assembly_supply: A1__DC1__amt <= 13000

A2__assembly_supply: A2__DC1__amt <= 3500

DC1__conservation_of_flow: A1__DC1__amt + A2__DC1__amt - DC1__R1__amt
 - DC1__R2__amt - DC1__R3__amt = 0

VARIABLES
A1__DC1__amt Continuous
A2__DC1__amt Continuous
DC1__R1__amt Continuous
DC1__R2__amt Continuous
DC1__R3__amt Continuous



In [7]:
# Show the outputs
my_model.show_outputs()

{'objective': 115872.32,
 'status': 'Optimal',
 'variables': {'A1__DC1__amt': 6646.0,
               'A2__DC1__amt': 3500.0,
               'DC1__R1__amt': 2500.0,
               'DC1__R2__amt': 4350.0,
               'DC1__R3__amt': 3296.0}}
